# Automatización de separación de PDF por radicado y empresa

Este notebook automatiza la separación de un PDF masivo de notificaciones, generando un archivo PDF por cada correo, usando el código de radicado y el nombre de la empresa en el nombre del archivo.

## 1. Instalación de librerías necesarias

Esta celda instala las librerías requeridas para manipular PDFs y extraer texto.

In [6]:
# Instala PyPDF2 y pdfplumber si no están instalados
# !pip install PyPDF2 pdfplumber

## 2. Importación de librerías y definición de funciones

Esta celda importa las librerías necesarias y define la función para separar el PDF según el radicado y empresa.

In [7]:
import os
import re
import pdfplumber
from PyPDF2 import PdfWriter

# Función para extraer radicado y Nit/CC de una página
def extraer_info(texto):
    radicado_match = re.search(r'(AEI/\d{3}\. \d{5})', texto)
    radicado = radicado_match.group(1) if radicado_match else None
    if radicado:
        radicado = radicado.replace('/', '-')
    nit_match = re.search(r'(Nit / CC \d+)', texto)
    nit = nit_match.group(1).replace(' ', '_') if nit_match else None
    return radicado, nit

# Función principal para separar el PDF por bloques de páginas
def separar_pdf_por_radicado(pdf_path, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    with pdfplumber.open(pdf_path) as pdf:
        bloques = []
        bloque_actual = {'clave': None, 'paginas': []}
        for i, page in enumerate(pdf.pages):
            texto = page.extract_text() or ''
            radicado, nit = extraer_info(texto)
            clave = f'{radicado}_{nit}' if radicado and nit else None
            # Si encontramos un nuevo bloque (nuevo radicado/nit)
            if clave and (bloque_actual['clave'] != clave):
                if bloque_actual['clave'] and bloque_actual['paginas']:
                    bloques.append(bloque_actual)
                bloque_actual = {'clave': clave, 'paginas': [i]}
            else:
                bloque_actual['paginas'].append(i)
        # Agregar el último bloque
        if bloque_actual['clave'] and bloque_actual['paginas']:
            bloques.append(bloque_actual)

    from PyPDF2 import PdfReader
    reader = PdfReader(pdf_path)
    for bloque in bloques:
        writer = PdfWriter()
        for idx in bloque['paginas']:
            writer.add_page(reader.pages[idx])
        nombre_pdf = f"{bloque['clave']}.pdf".replace('/', '-')
        ruta_pdf = os.path.join(output_dir, nombre_pdf)
        with open(ruta_pdf, 'wb') as f_out:
            writer.write(f_out)
        print(f"Se guardó: {ruta_pdf} con {len(bloque['paginas'])} páginas")

## 3. Ejecución de la automatización

Esta celda ejecuta la función principal para separar el PDF masivo y guardar los archivos individuales en la carpeta de resultados.

In [8]:
# Ruta del PDF masivo y carpeta de salida
pdf_path = r'C:\Users\osmarrincon\Downloads\pdf\NOTIFICACION  UGPP 1.pdf'
output_dir = r'C:\Users\osmarrincon\Downloads\pdf\Resultado'

# Ejecutar la separación
separar_pdf_por_radicado(pdf_path, output_dir)

Se guardó: C:\Users\osmarrincon\Downloads\pdf\Resultado\AEI-120. 37889_Nit_-_CC_9008063579.pdf con 3 páginas
Se guardó: C:\Users\osmarrincon\Downloads\pdf\Resultado\AEI-120. 37888_Nit_-_CC_9008063579.pdf con 3 páginas
Se guardó: C:\Users\osmarrincon\Downloads\pdf\Resultado\AEI-120. 37887_Nit_-_CC_1123038700.pdf con 3 páginas
Se guardó: C:\Users\osmarrincon\Downloads\pdf\Resultado\AEI-120. 37886_Nit_-_CC_1123038700.pdf con 3 páginas
Se guardó: C:\Users\osmarrincon\Downloads\pdf\Resultado\AEI-120. 37885_Nit_-_CC_1118650907.pdf con 3 páginas
Se guardó: C:\Users\osmarrincon\Downloads\pdf\Resultado\AEI-120. 37884_Nit_-_CC_1118650907.pdf con 3 páginas
Se guardó: C:\Users\osmarrincon\Downloads\pdf\Resultado\AEI-120. 37883_Nit_-_CC_1118650442.pdf con 3 páginas
Se guardó: C:\Users\osmarrincon\Downloads\pdf\Resultado\AEI-120. 37882_Nit_-_CC_1118648151.pdf con 3 páginas
Se guardó: C:\Users\osmarrincon\Downloads\pdf\Resultado\AEI-120. 37881_Nit_-_CC_1118648151.pdf con 3 páginas
Se guardó: C:\Users